# Using a Python implementation of the Flickr API

This notebook uses the [flickr_api package](https://github.com/alexis-mignon/python-flickr-api/) and the BL Digital Research team's API keys. Provides access to the Albums (photosets) available on our account, the tags on those photos, and the [DETR-Resnet-50](https://huggingface.co/facebook/detr-resnet-50) model for detecting objects in images.

In [ ]:
import sys
if "../" not in sys.path:
    sys.path.append("../")
import random
import os

import flickr_api
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import requests
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch

In [ ]:
# a = flickr_api.auth.AuthHandler() # creates a new AuthHandler object
# perms = "read" # set the required permissions
# url = a.get_authorization_url(perms)
# print(url) # this is the url we need!

# a.set_verifier("8638b6801bd2ac7e")
# flickr_api.set_auth_handler(a)

# a.save("..\\flickr_api_session.txt")

In [ ]:
flickr_api.set_auth_handler("..\\flickr_api_session_auth.txt") # or whatever you save your auth file as

In [ ]:
user = flickr_api.Person.findByEmail("digitalresearch@bl.uk")

In [ ]:
photosets = []

for x in range(1, 4):
    photosets.extend(user.getPhotosets(page=x))

In [ ]:
# info = []
# for ps in photosets:
#     try:
#         info.append(ps.getInfo())
#     except flickr_api.flickrerrors.FlickrServerError:
#         info.append(None)
#     except TimeoutError:
#         info.append(None)

# info_df = pd.DataFrame.from_records(info).set_index("id")
# info_df.to_csv("..\\data\\processed\\photoset_info.csv")

## Info on all the Albums (=photosets) available on the BL account

In [ ]:
info_df = pd.read_csv("..\\data\\processed\\photoset_info.csv", index_col="id")

In [ ]:
info_df.head(5)

In [ ]:
# LwM
lwm_ps = photosets[0].getPhotos()

In [ ]:
# Women of the World
wow_ps = photosets[2].getPhotos()

In [ ]:
# Book Illustrations
book_illus_ps = photosets[4].getPhotos(page=1)

## Tags on BL photos

Using the Women of the World album as an example

In [ ]:
# wow_tags = [x.getTags() for x in wow_ps]
# wow_tags_text = {ps.getPhotoUrl():[x.text for x in ts] for ts, ps in zip(wow_tags, wow_ps)}
# photo_ids = [os.path.basename(x[:-9]) for x in list(wow_tags_text.keys())]
# wow_tags_idxed = {id: tags for id, tags in zip(photo_ids, wow_tags_text.values())}
# wow_tags_dfs = [pd.DataFrame(data={"id":x, "tag":tags}) for x, tags in enumerate(wow_tags_idxed.values())]
# wow_tags_df = pd.concat(wow_tags_dfs)
# # wow_tags_df.to_csv("..\\data\\processed\\wow_tags.csv", index=False)

In [ ]:
wow_tags_df = pd.read_csv("..\\data\\processed\\wow_tags.csv")

In [ ]:
wow_tags_df

In [ ]:
# sherlock_tags_df = wow_tags_df[wow_tags_df["tag"].str.contains("sherlocknet")].reset_index()

# with open("..\\data\\external\\ofcom_bad_words.txt", "r") as f:
#     bad_words = f.readlines()
#     bad_words = [x.strip("\n").lower() for x in bad_words][1:]

# sherlock_tags_df["tag_val"] = sherlock_tags_df["tag"].apply(lambda x: x.split("=")[1])
# sherlock_tags_df["bad_word"] = sherlock_tags_df["tag_val"].apply(lambda x: x in bad_words)

# sherlock_tags_df[sherlock_tags_df["bad_word"]]["tag_val"].unique()

e.g. BL URL to help find the photo a tag is attached to  
https://www.flickr.com/photos/britishlibrary/11244481994

## DETR-Resnet-50

Using the DETR-Resnet-50 model created by Facebook to try detecting objects in images from your chosen album

In [ ]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

In [ ]:
def to_rectangle(xy, w, h):
    c = "#{:06x}".format(int(random.random() * 0xffffff))
    p = Rectangle(xy, w, h, ec=c, fill=False)
    return p

def draw_labelled_box(ax, box, label, score):
    xy = (box[0], box[1])
    w, h = box[2] - box[0], box[3] - box[1]
    rct = to_rectangle(xy, w, h)
    ax.add_patch(rct)
    ax.text(box[0] + 0.01*w, box[1] + 0.03*h, s=f"{label} {score}", backgroundcolor="white", va='top')

In [ ]:
url = wow_ps[0].getPhotoFile(size_label="Medium")
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)[0]

In [ ]:
fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"frameon": False})
ax.imshow(image)
ax.set_axis_off()

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    label_text = model.config.id2label[label.item()]
    score_val = round(score.item(), 3)
    print(f"Detected {label_text} with confidence {score_val} at location {box}")
    draw_labelled_box(ax, box, label_text, score_val)